https://data-science-blog.com/blog/2022/04/11/how-to-choose-the-best-pre-trained-model-for-your-convolutional-neural-network/
https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/

In [3]:
%pip install fast_ml

Note: you may need to restart the kernel to use updated packages.Installing collected packages: fast-ml



In [93]:
%pip install -U efficientnet


Note: you may need to restart the kernel to use updated packages.


In [1]:
# import the necessary packages
import os
import zipfile
import matplotlib as plt
import tensorflow as tf
import cv2

# from pyimagesearch.resnet import ResNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
#from keras import optimizers
from tensorflow.keras import optimizers

from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
from matplotlib import pyplot
import efficientnet.keras as efn

from tensorflow.keras.applications.vgg16 import VGG16


In [17]:
# load the image
#img = load_img('data/IMG_3559.jpg')

def image_augmentation(data):
    new_data = []
    # convert to numpy array
    for i in range(len(data)):
        temp_data = img_to_array(data[i])
        # normalize image
        # print("data: ", data)
        # normalize image
        img_norm = cv2.normalize(temp_data, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        # print("norm: ", img_norm)
        # expand dimension to one sample
        samples = expand_dims(img_norm, 0)
        # create image data augmentation generator

        
        datagen = ImageDataGenerator(rescale = 1./255., width_shift_range = 0.2, height_shift_range = 0.2, horizontal_flip = True ,rotation_range=30, brightness_range=[0.2,1.2], zoom_range=[0.6,1.1])
        # prepare iterator
        it = datagen.flow(samples, batch_size=1)
        new_data.append(it)
        
        # generate samples and plot
        for i in range(4):
            # generate batch of images
            batch = it.next()
            # convert to unsigned integers for viewing
            image = batch[0].astype('uint8')
            im = array_to_img(image, scale=False)
            im.save('/tmp/Datensatz/Datensatz/image_DA_'+str(i)+ '.jpg')

            # pyplot.imshow(image)
            # # show the figure
            # pyplot.show()
    #train_generator = datagen.flow_from_directory(new_data, batch_size = 20, class_mode = 'binary', target_size = (224, 224))
    return new_data

In [2]:
################ Unzip the dataset in the Colab runtime #################
import glob
import zipfile
zip_file_path = "data/Datensatz.zip"

pathToZip = os.path.join(zip_file_path)

print("unzipping {}".format(pathToZip))

pathToData = "/tmp/Datensatz"
with zipfile.ZipFile(pathToZip, 'r') as zip_ref:
    zip_ref.extractall(pathToData)

# training images
data    = glob.glob(os.path.join(pathToData, "Datensatz/" + "*.jpeg"))
# test images
#test    = glob.glob(os.path.join(pathToData, "Datensatz/test/" + "*.jpg"))

print(pathToData + "/Datensatz/") #/train
print("train    : {} image paths".format(len(data)))
#print("train    : {} image paths".format(len(test)))


unzipping data/Datensatz.zip
/tmp/Datensatz/Datensatz/
train    : 535 image paths


In [3]:
from PIL import Image
import glob
image_list = []
for filename in glob.glob("/tmp/Datensatz/Datensatz/" + "*.jpeg"): #assuming gif
    im=Image.open(filename)
    image_list.append(im)

print("list: ", len(image_list))
print(image_list[0])

list:  535
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2316x2316 at 0x1624602CD30>


In [10]:
import numpy as np
samples = []
for i in range(len(image_list)):
        temp_data = img_to_array(image_list[i])
        # normalize image
        # print("data: ", data)
        # normalize image
        img_norm = cv2.normalize(temp_data, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        # print("norm: ", img_norm)
        # expand dimension to one sample
        samples.append(expand_dims(img_norm.astype(np.uint8), 0))

MemoryError: Unable to allocate 105. MiB for an array with shape (3024, 3024, 3) and data type float32

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8
y = []

for i in range(len(samples)):
    y.append('Dominik')


# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(samples,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train), print(y_train)
print(X_valid), print(y_valid)
print(X_test), print(y_test)

ValueError: With n_samples=1, test_size=None and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [7]:
for i in range(316, len(X_train)):
    X_train[i].save(r'C:\Users\domin\Documents\Studium\Master\Deep-Learning\Projekt\face_recognition\data\data\Dominik\train\image_' +str(i) +'.jpg', 'JPEG')
    
for i in range(len(X_valid)):
    X_train[i].save(r'C:\Users\domin\Documents\Studium\Master\Deep-Learning\Projekt\face_recognition\data\data\Dominik\validation\image_' +str(i) +'.jpg', 'JPEG')

for i in range(len(X_test)):
    X_train[i].save(r'C:\Users\domin\Documents\Studium\Master\Deep-Learning\Projekt\face_recognition\data\data\Dominik\test\image_' +str(i) +'.jpg', 'JPEG')
    

In [7]:
import pandas as pd

df_train_path = pd.DataFrame (X_train, columns = ['image_path'])
df_valid_path = pd.DataFrame (X_valid, columns = ['image_path'])
df_test_path = pd.DataFrame (X_test, columns = ['image_path'])

df_train_label = pd.DataFrame (y_train, columns = ['label'])
df_valid_label = pd.DataFrame (y_valid, columns = ['label'])
df_test_label = pd.DataFrame (y_test, columns = ['label'])

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255., width_shift_range = 0.2, height_shift_range = 0.2, horizontal_flip = True ,rotation_range=70, brightness_range=[0.2,1.2], zoom_range=[0.6,1.1], fill_mode='reflect')
test_datagen = ImageDataGenerator(rescale = 1./255.)

path_train = "/tmp/face_images/face_images/train"
path_test = "/tmp/face_images/face_images/test"
# train_generator = train_datagen.flow_from_directory(path_train, batch_size = 20, class_mode = 'binary', target_size = (224, 224), save_to_dir= 'data/new2/')
# test_generator = test_datagen.flow_from_directory(path_test, batch_size = 20, class_mode = 'binary', target_size = (224, 224), save_to_dir= 'data/new2/')

train_generator = train_datagen.flow(x=X_train, y=y_train, batch_size = 20, save_to_dir= 'data/split_data/train/')
test_generator = test_datagen.flow(x=df_valid_path,y=df_valid_label, batch_size = 20, save_to_dir= 'data/split_data/validation/')
test_generator = test_datagen.flow(x=df_test_path,y=df_test_label, batch_size = 20, save_to_dir= 'data/split_data/test/')


TypeError: object of type 'JpegImageFile' has no len()

In [9]:
# Step 2: Loading the Base Model
# sampleImage = []
# for i in range(len(train)):
#     sampleImage.append(cv2.imread(train[i]))
#print("sampleImage: ", sampleImage[2])
#train_dat, train_generator = image_augmentation(sampleImage)

base_model = efn.EfficientNetB7(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

Found 20 images belonging to 2 classes.


In [4]:
# Again, let us freeze the layers:

for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Step 3: Build the model

x = base_model.output
y = base_model.input
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(y, predictions) #base_model.input

In [13]:
# Step 4: Compile and Fit

model_final.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

c:\Users\domin\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Batch-Size: Think of a batch as a for-loop iterating over one or more samples and making predictions. At the end of the batch, the    predictions are compared to the expected output variables and an error is calculated. From this error, the update algorithm is used to improve the model, e.g. move down along the error gradient.
-What if the dataset does not divide evenly by the batch size?
    --> This can and does happen often when training a model. It simply means that the final batch has fewer samples than the other batches.

Epochen: The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset.

One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches. For example, as above, an epoch that has one batch is called the batch gradient descent learning algorithm.

You can think of a for-loop over the number of epochs where each loop proceeds over the training dataset. Within this for-loop is another nested for-loop that iterates over each batch of samples, where one batch has the specified “batch size” number of samples.

The batch size is a number of samples processed before the model is updated.

The number of epochs is the number of complete passes through the training dataset.


Steps_per_epoch: Total number of steps (batches of samples)
            before declaring one epoch finished and starting the
            next epoch. When training with input tensors such as
            TensorFlow data tensors, the default `None` is equal to
            the number of samples in your dataset divided by
            the batch size, or 1 if that cannot be determined. If x is a
            `tf.data` dataset, and 'steps_per_epoch'
            is None, the epoch will run until the input dataset is exhausted.
            When passing an infinitely repeating dataset, you must specify the
            `steps_per_epoch` argument.

In [20]:
# We finally fit the model on our data:

eff_history = model_final.fit(train_generator, validation_data = train_generator, steps_per_epoch = 2, epochs = 10)

Epoch 1/10


TypeError: 'NoneType' object is not callable

In [ ]:
# Importing OpenCV package
import cv2
  
# Reading the image
img = cv2.imread('Photos/cric4.jpg')
  
# Converting image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Loading the required haar-cascade xml classifier file
haar_cascade = cv2.CascadeClassifier('Haarcascade_frontalface_default.xml')
  
# Applying the face detection method on the grayscale image
faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9)
  
# Iterating through rectangles of detected faces
for (x, y, w, h) in faces_rect:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
  
cv2.imshow('Detected faces', img)
  
cv2.waitKey(0)